# Capstone Project Part 1 - ETL and Data Modeling

During this capstone project, you will develop a data pipeline as part of a new project in the company DeFtunes. You will showcase the abilities and tools you have been using during the whole specialization.

# Table of Contents

- [ 1 - Introduction](#1)
- [ 2 - Data Sources](#2)
- [ 3 - Exploratory Data Analysis](#3)
- [ 4 - ETL Pipeline with AWS Glue and Terraform](#4)
  - [ 4.1 - Landing Zone](#4.1)
  - [ 4.2 - Transformation Zone](#4.2)
  - [ 4.3 - Serving Zone](#4.3)
- [ 5 - Data Modeling with dbt and Redshift Spectrum](#5)
  - [ 5.1 - Redshift Setup](#5.1)
  - [ 5.2 - Redshift Test](#5.2)
  - [ 5.3 - dbt Setup](#5.3)
  - [ 5.4 - Data Modeling](#5.4)
- [ 6 - Upload Files for Grading](#6)

<a name='1'></a>
## 1 - Introduction

DeFtunes is a new company in the music industry, offering a subscription-based app for streaming songs. Recently, they have expanded their services to include digital song purchases. With this new retail feature, DeFtunes requires a data pipeline to extract purchase data from their new API and operational database, enrich and model this data, and ultimately deliver the comprehensive data model for the Data Analysis team to review and gain insights. Your task is to develop this pipeline, ensuring the data is accurately processed and ready for in-depth analysis.

Here is the diagram with the main requirements for this project:

![Capstone_Diagram](images/Capstone-diagram.png)

1. The pipeline has to follow a medallion architecture with a landing, transform and serving zone.
2. The data generated in the pipeline will be stored in the company's data lake, in this case, an S3 bucket.
3. The silver layer should use Iceberg tables, and the gold layer should be inside Redshift.
4. The pipeline should be reproducible, you will have to implement it using Terraform.
5. The data should be modelled into a star schema in the serving layer, you should use dbt for the modelling part.

Before starting, you will need to import some required libraries and modules for the capstone development.

In [ ]:
import json
import requests
import pandas as pd
import subprocess

%load_ext sql

LAB_PREFIX='de-c4w4a1'

<a name='2'></a>
## 2 - Data Sources

The first data source you will be using is the DeFtunes operational database, which is running in RDS with a Postgres engine. This database contains a table with all the relevant information for the available songs that you can purchase. Let's connect to the table using the `%sql` magic. 

2.1. To define the connection string, go to CloudFormation Outputs in the AWS console. You will see the key `PostgresEndpoint`, copy the corresponding **Value** and replace with it the placeholder `<POSTGRES_ENDPOINT>` in the cell below (please, replace the whole placeholder including the brackets `<>`). Then run the cell code.

In [ ]:
RDSDBHOST = '<POSTGRES_ENDPOINT>'
RDSDBPORT = '5432'
RDSDBNAME = 'postgres'
RDSDBUSER = 'postgresuser'
RDSDBPASSWORD = 'adminpwrd'

postgres_connection_url = f'postgresql+psycopg2://{RDSDBUSER}:{RDSDBPASSWORD}@{RDSDBHOST}:{RDSDBPORT}/{RDSDBNAME}'
%sql {postgres_connection_url}

2.2. Test that the connection works by running the following query.

In [ ]:
%%sql
SELECT schema_name
FROM information_schema.schemata;

2.3. Inside the `deftunes` schema there is a table `songs` which was mentioned before. Let's query a sample from it:

*Note:* The `songs` table is based on the Million Song Dataset, more information can be found [here](http://millionsongdataset.com/).

In [ ]:
%%sql
SELECT *
FROM deftunes.songs
LIMIT 5;

2.4. The second data source is a new API designed for the song purchase process. This API contains information on the purchases done by the users of the App and also contains relevant information about each user. Copy the endpoint value from the CloudFormation outputs tab and replace the placeholder `<API_ENDPOINT>` with it.

In [ ]:
API_ENDPOINT = "<API_ENDPOINT>"

You can also access the documentation to the API by opening the new browser tab, pasting the API endpoint value and adding `/docs` to it. You will see an interactive interface to test the API.

2.5. The first endpoint is to the `sessions` path in the API, which retrieves the transactional sessions. Let's test the API by performing a GET request to the endpoint with the next cell. If everything works you should get a `200` status code from the `sessions_response` object.

In [ ]:
request_start_date = "2020-01-01"
request_end_date = "2020-03-01"
sessions_response = requests.get(f'http://{API_ENDPOINT}/sessions?start_date={request_start_date}&end_date={request_end_date}')
print(sessions_response.status_code)

2.6. You can get the content of the response in JSON format using the `.json()` method, let's print the first record with the following cell.

In [ ]:
sessions_json = sessions_response.json()
print(json.dumps(sessions_json[0], indent=4))

2.7. The second endpoint is to the `users` path in the API, it retrieves the transactional sessions. Perform a GET request to the endpoint with the next cell, then print a sample with the cell after that one.

In [ ]:
users_request = requests.get(f'http://{API_ENDPOINT}/users')
print(users_request.status_code)

In [ ]:
users_json = users_request.json()
print(json.dumps(users_json[0], indent=4))

<a name='3'></a>
## 3 - Exploratory Data Analysis

To better understand the data sources, start analyzing the data types and values that come from each source. You can use the pandas library to perform Exploratory Data Analysis (EDA) on samples of data.

3.1. Let's begin with the `songs` table in the source RDS database, we will take advantage of the `%sql` magic to bring a sample with SQL and convert it into a pandas dataframe.

In [ ]:
songs_result = %sql SELECT *FROM deftunes.songs LIMIT 5
songs_df = songs_result.DataFrame()
songs_df.head()

3.1. Use Pandas info() method to print out a summary of information about the dataframe, including information about the columns such as their data types.

In [ ]:
print(songs_df.info())

3.2. Use the describe() method to generate a summary of statistics about the numerical and object columns in the dataframe.

In [ ]:
songs_df.describe()

3.3. Convert JSON objects `sessions_json` and `users_json` into pandas dataframes, and display the first few rows.

In [ ]:
session_df = pd.json_normalize(sessions_json)
session_df.head()

In [ ]:
user_df = pd.json_normalize(users_json)
user_df.head()

<a name='4'></a>
## 4 - ETL Pipeline with AWS Glue and Terraform

Now you will start creating the required resources and infrastructure for your data pipeline. Remember that you will use a medallion architecture.

The pipeline will be composed by the following steps:
- An extraction job to get the data from the PostgreSQL Database. This data will be stored in the landing zone of your Data Lake.
- An extraction job to get the data from the two API endpoints. This data will be stored in the landing zone of your Data Lake in JSON format.
- A transformation job that takes the raw data extracted from the PostgreSQL Database, casts some fields to the correct data types, adds some metadata and stores the dataset in Iceberg format.
- A transformation that takes the JSON data extracted from the API endpoints, normalizes some nested fields, adds metadata and stores the dataset in Iceberg format.
- The creation of some schemas in your Data Warehouse hosted in Redshift.

<a name='4.1'></a>
### 4.1 - Landing Zone

For the landing zone, you are going to create three Glue Jobs: one to extract the data from the PostgreSQL database and two to get the data from each API's endpoint. You are going to create those jobs using Terraform to guarantee that the infrastructure for each job will be always the same and changes can be tracked easily. Let's start by creating the jobs and then creating the infrastructure.

4.1.1. Go to the `terraform/assets/extract_jobs` folder. You will find two scripts

- `de-c4w4a1-extract-songs-job.py`: this script extracts data from the PostgreSQL source database.
- `de-c4w4a1-api-extract-job.py`: this script extracts data from the API. Endpoints can be provided through parameters.

Open each of them and follow the instructions in the comments to complete the scripts. Save changes to both of the files.

In a later section, you will run those Glue Jobs. Take into account that in the landing zone of your Data Lake you will see that the data will be stored in subfolders named according to the date of ingestion, which by default is your current date.

4.1.2. You will need to complete the terraform module `extract_job`. Given that you already created the scripts for the Glue Jobs, let's start by uploading them to an S3 bucket. Open the `terraform/modules/extract_job/s3.tf` file. You are already provided with some resources such as the scripts bucket and its permissions. Complete the code in the file following the instructions. Make sure that you save changes.

4.1.3. Open the `terraform/modules/extract_job/glue.tf` file. In this file you will set all the required resources to create the Glue Jobs. Complete the code following the instructions and save changes.

4.1.4. Explore the rest of the files of the `extract_job` module to understand the whole infrastructure. Avoid performing further changes to other files. Here is the summary of what you can find in those files:
- In `terraform/modules/extract_job/iam.tf` file you can find the creation of the role used to execute the Glue Jobs. There is also the attachment of a policy holding the permissions. Those permissions can be found directly in the `terraform/modules/extract_job/policies.tf` file.
- The `terraform/modules/extract_job/network.tf` has the definition of the private subnet and the source database security group used to create the Glue Connection used to allow the Glue Jobs to connect to the source PostgreSQL database.
- The `terraform/modules/extract_job/variables.tf` contains the necessary input parameters for this module, while the `terraform/modules/extract_job/outputs.tf` sets the possible outputs that terraform will show in the console from this module.

4.1.5. You are ready to deploy the first module of your infrastructure. Open the `terraform/main.tf` file and uncomment the lines associated with the module named `extract_job` (lines 1 to 14); make sure to keep the rest of modules commented. Open the `terraform/outputs.tf` file and uncomment the outputs associated with the extract module (lines 5 to 20). Save changes in both of the files.

4.1.6. In the VSCode terminal, go to the `terraform` folder and deploy the infrastructure with the following commands.

*Note*:<span style="color:red"> All terminal commands in this lab should be run in the VSCode terminal, not Jupyter, as it may cause some issues.</span>

```bash
cd terraform
terraform init
terraform plan
terraform apply
```

*Note*: Remember that the command `terraform apply` will prompt you to reply `yes`.

4.1.7. You will get some outputs, in particular, you require the following three: `glue_api_users_extract_job`, `glue_sessions_users_extract_job` and `glue_rds_extract_job`. Those outputs correspond to the three glue jobs that will extract the data from the API endpoints and the database respectively. Use the following command in the terminal to execute each job based on its name. Replace `<JOB-NAME>` with the value of the terraform outputs (`de-c4w4a1-api-users-extract-job`, `de-c4w4a1-api-sessions-extract-job` and `de-c4w4a1-rds-extract-job`) respectively. You can run those three jobs in parallel.

```bash
aws glue start-job-run --job-name <JOB-NAME> | jq -r '.JobRunId'
```

You should get `JobRunID` in the output. Use this job run ID to track each job status by using this command, replacing the `<JOB-NAME>` and `<JobRunID>` placeholders.

```bash
aws glue get-job-run --job-name <JOB-NAME> --run-id <JobRunID> --output text --query "JobRun.JobRunState"
```

Wait until the statuses of those three jobs change to `SUCCEEDED` (each job should take around 3 mins).

<a name='4.2'></a>
### 4.2 - Transformation Zone

Once you have run the jobs that feed the first layer of your Data Lake three-tier architecture, it is time to generate the jobs to transform the data and store it in the second layer. The methodology will be similar to the previous zone: you will create the Glue Job scripts to take the data out of the landing layer, transform it and put it into the transformation zone layer. Then you will create the necessary resources in AWS using Terraform.

4.2.1. Go to the `terraform/assets/transform_jobs` folder. You will find two scripts:
- `de-c4w4a1-transform-songs-job.py`
- `de-c4w4a1-transform-json-job.py`

The two scripts will take the data out of the `landing_zone` layer in the Data Lake and, after some transformations, will store the data into the `transform_zone` layer in Apache Iceberg format.

Open each of them and follow the instructions in the comments to complete the scripts. Save changes to both of the files.

If you want to know more about saving data into Apache Iceberg format using the Glue Catalog, you can check the [documentation](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-programming-etl-format-iceberg.html#aws-glue-programming-etl-format-iceberg-insert).

4.2.2. Time to complete the terraform module `transform_job`. Open the `terraform/modules/transform_job/s3.tf` file. Complete the script following the instructions and save changes.

4.2.3. Open the `terraform/modules/transform_job/glue.tf` file. Complete the code following the instructions and save changes.

4.2.4. Open the `terraform/main.tf` file and uncomment the lines associated with the module named `transform_job` (lines 16 to 30). 

4.2.5. Open the `terraform/outputs.tf` file and uncomment the lines 22 to 34. 

4.2.6. In the terminal, go to the `terraform` folder and deploy the infrastructure with the following commands:

```bash
cd terraform
terraform init
terraform plan
terraform apply
```

*Note*: Remember that the command `terraform apply` will prompt you to reply `yes`.

4.2.7. You will get some additional outputs, in particular, you require the following two: `glue_json_transformation_job` and `glue_songs_transformation_job`. Execute the two glue jobs, based on the name (`de-c4w4a1-json-transform-job` or `de-c4w4a1-songs-transform-job`). You can run those two jobs in parallel. Use the following command in the VSCode terminal:

```bash
aws glue start-job-run --job-name <JOB-NAME> | jq -r '.JobRunId'
```

And based on the job run ID track each job status by using this command:

```bash
aws glue get-job-run --job-name <JOB-NAME> --run-id <JobRunID> --output text --query "JobRun.JobRunState"
```

Wait until the jobs statuses change to `SUCCEEDED` (each job should take around 3 mins).

<a name='4.3'></a>
### 4.3 - Serving Zone

For the last layer of your Three-tier Data Lake architecture, you are going to use AWS Redshift as a Data Warehouse solution. The transformations will be performed directly inside Redshift, but you need to make the data available in that storage solution. For that, you will use Redshift Spectrum, which is a feature that allows you to run queries against data stored in S3 without having to load the data into Redshift tables. For that, you are required to use a Glue Catalog which was already created in the `transform` module.

Follow the instructions below to finish setting up your resources for the `serving` module.

4.3.1. Open the file located at `terraform/modules/serving/iam.tf`. Complete the code and save changes.

4.3.2. Open the file `terraform/modules/serving/redshift.tf`. Complete the code and save changes.

4.3.4. Open the `terraform/main.tf` file and uncomment the lines associated with the module named `serving` (lines 32 to 48). 

4.3.5. Uncomment the corresponding lines in the `terraform/outputs.tf` file (lines 37 to 44).  

4.3.6. Deploy the infrastructure for the last layer. In the terminal, go to the `terraform` folder and run the commands:

```bash
cd terraform
terraform init
terraform plan
terraform apply
```

*Note*: Remember that the command `terraform apply` will prompt you to reply `yes`.

With that, you have deployed the required infrastructure for your three-tier data lake. The next step consists of modelling the data in your Redshift Data Warehouse to serve it.

<a name='5'></a>
## 5 - Data Modeling with dbt and Redshift Spectrum

<a name='5.1'></a>
### 5.1 - Redshift Setup

Before working with DBT to model the data in the transformation layer into the serving layer, you need to use **Redshift Spectrum** to connect Redshift with the Iceberg tables. Spectrum allows you to query files from S3 directly from Redshift, it also has a special feature that allows us to read directly from Iceberg tables just by creating an external schema that points to the Glue database containing the tables. For this initial setup, you will use Terraform to set up the external schema and a normal schema for the serving layer.

5.1.1. Navigate to the `terraform` folder and run the serving module with the following command:

```bash
terraform apply -target=module.serving
```

<a name='5.2'></a>
### 5.2 - Redshift Test

To verify that the schemas were set up successfully, you will connect to the target Redshift cluster using the `%sql` magic. 

5.2.1. Let's start by configuring the credentials, you can obtain the Redshift's cluster endpoint in the CloudFormation stack's outputs and replace the placeholder `<REDSHIFT_ENDPOINT>` with it.

In [ ]:
REDSHIFTDBHOST = '<REDSHIFT_ENDPOINT>'
REDSHIFTDBPORT = 5439
REDSHIFTDBNAME = 'dev'
REDSHIFTDBUSER = 'defaultuser'
REDSHIFTDBPASSWORD = 'Defaultuserpwrd1234+'

redshift_connection_url = f'postgresql+psycopg2://{REDSHIFTDBUSER}:{REDSHIFTDBPASSWORD}@{REDSHIFTDBHOST}:{REDSHIFTDBPORT}/{REDSHIFTDBNAME}'
%sql {redshift_connection_url}

5.2.2. Test the connection and the setup, this query will return the list of available schemas for the `dev` database, and the external schema and gold layer schema should appear.

In [ ]:
%sql SHOW SCHEMAS FROM DATABASE dev 

5.2.3. Now, let's verify that the Iceberg tables where automatically imported into the external schema, let's query the tables available inside the external schema.

In [ ]:
%sql SHOW TABLES FROM SCHEMA dev.deftunes_transform

Query the Iceberg tables in the external schema to verify that Redshift can read from them.

In [ ]:
%sql select * from deftunes_transform.songs limit 10

In [ ]:
%sql select * from deftunes_transform.sessions limit 10

In [ ]:
%sql select * from deftunes_transform.users limit 10

<a name='5.3'></a>
### 5.3 - dbt Setup

Now that you have set up the target database in Redshift, you will create a dbt project that connects to Redshift and allows you to model the transform layer tables into the final data model in the serving layer. 

5.3.1. Create the new project using the following commands in the VSCode terminal.

*Note*:<span style="color:red"> All terminal commands in this lab should be run in the VSCode terminal, not Jupyter, as it may cause some issues. Always check that the virtual environment is active.</span>

```bash
cd ..
source jupyterlab-venv/bin/activate
dbt init dbt_modeling
```

After running the command, dbt will ask you the engine to run the project on, select the option for Redshift. The CLI will ask you for the connection details, use the same connection values you used before to configure the `%sql` magic (step 5.2.1), when asked for the `dbname` input `dev`, for `schema` input `deftunes_serving`, for `threads` input `1`. 

![dbt setup](images/dbt_config.png)

5.3.2. To test the connection, run the following commands:

```bash
cd dbt_modeling
dbt debug
```

If everything was correctly configured, you should see the following text at the end of the output:

```bash
Connection test: [OK connection ok]
```

*Note*: If you had issues defining the connection details, you can use the `profiles.yml` file in the scripts folder as a guide to define the connection details, change the placeholder in the file with the Redshift cluster endpoint and then copy it to the following path `~/.dbt/profiles.yml ` with this command:

```bash
cp ../scripts/profiles.yml ~/.dbt/profiles.yml 
```

<a name='5.4'></a>
### 5.4 - Data Modeling

*Note*: This section is optional and not graded.

5.4.1. Now that the dbt project has the initial setup, create a new folder named `serving_layer` in the models folder, this subfolder will contain the models associated with the star schema.
```bash
cd models
mkdir serving_layer
```

5.4.2. In the `./dbt_modeling/dbt_project.yml`, change the `models` block to the following one:

```yaml
models:
  dbt_modeling:
    serving_layer:
      +materialized: table
```

Save changes to the file.

5.4.3. Now you can prepare files for data modeling into the star schema. You will need to identify fact and dimensional tables, then create an SQL model file for each. Finally, inside the new folder, create a `schema.yml` file. You can look at the `example` folder if needed. Once you are done modelling the data, use the following command to run the models you created (make sure you are in the `dbt_modeling` project folder):

```bash
dbt run -s serving_layer
```

If all the model runs were successful, you should see an output like this one, where N is the number of models you created.
```bash
Completed successfully

Done. PASS=X WARN=0 ERROR=0 SKIP=0 TOTAL=X
```

5.4.4. The final test for your models will be for you to query them using the `%sql` magic. Run the following query for each table to get a sample and verify that the model definition was correct (replace the placeholder `<TABLE_NAME>`).

In [ ]:
%sql SHOW TABLES FROM SCHEMA dev.deftunes_serving

In [ ]:
%sql SELECT * FROM deftunes_serving.<TABLE_NAME> limit 10

In [ ]:
%sql DROP SCHEMA deftunes_serving CASCADE;

During the first part of the capstone, you set up a data architecture for the new business operation of DeFtunes, you implemented a basic data pipeline that can be improved later on with an iterative approach. In the second part of the capstone, you will improve upon the existing data architecture adding orchestration, data visualization and data quality checks. 

<a name='6'></a>
## 6 - Upload Files for Grading

Upload the notebook into S3 bucket for grading purposes.

*Note*: you may need to click **Save** button before the upload.

In [ ]:
# Retrieve the AWS account ID
result = subprocess.run(['aws', 'sts', 'get-caller-identity', '--query', 'Account', '--output', 'text'], capture_output=True, text=True)
AWS_ACCOUNT_ID = result.stdout.strip()

SUBMISSION_BUCKET = f"{LAB_PREFIX}-{AWS_ACCOUNT_ID}-us-east-1-submission"

!aws s3 cp ./C4_W4_Assignment_1.ipynb s3://$SUBMISSION_BUCKET/C4_W4_Assignment_1_Learner.ipynb